In [15]:
import vtk

In [16]:
rd = vtk.vtkPolyDataReader()
rd.SetFileName('la.vtk')
rd.Update()
mesh = rd.GetOutput()

In [17]:
#Setup longitudinal problem
fe = vtk.vtkFeatureEdges()
fe.SetInputData(mesh)
fe.FeatureEdgesOff()
fe.BoundaryEdgesOn()
fe.NonManifoldEdgesOff()
fe.ManifoldEdgesOff()
fe.Update()

lines = fe.GetOutput()

In [18]:
with open('mesh.header','w') as f:
    f.write(f'{mesh.GetNumberOfPoints()} {mesh.GetNumberOfCells()} {lines.GetNumberOfCells()+15}\n')
    f.write('2\n')
    f.write(f'303 {mesh.GetNumberOfCells()}\n')
    f.write(f'202 {lines.GetNumberOfCells()+15}\n')
    

der file mesh.header tells h
(303) - triangle
202 - line
101 - point

nodes elements boundary-elements
nof_types
type-code nof_elements
type-code nof_elements
...
In the first line the numbers of nodes, elements, and boundary elements are given, while the count in the
second line is the number of different element types used in the mesh. The lines which follow give the
numbers of elements as sorted into different element types.
For example, the following header file
300 261 76
2
404 261
202 76
tells us that the mesh is composed of 300 nodes, 261 elements, and 76 boundary elements. Two different
element types are used in the mesh: there are 261 elements of type code 404 (bilinear quadrilateral) and 76
elements of type code 202 (linear line element).

In [19]:
with open('mesh.nodes','w') as f:
    for i in range(mesh.GetNumberOfPoints()):
        pt = mesh.GetPoint(i)
        f.write(f'{i+1} -1 {pt[0]} {pt[1]} {pt[2]}\n')

The file mesh.nodes contains node data so that each line defines one node. Each line starts with two
integers followed by three real numbers:
n1 -1 x y z
n2 -1 x y z
...
nn -1 x y z

In [20]:
with open('mesh.elements','w') as f:
    for i in range(mesh.GetNumberOfCells()):
        ids = mesh.GetCell(i).GetPointIds()
        f.write(f'{i+1} 1 303 {ids.GetId(0)+1} {ids.GetId(1)+1} {ids.GetId(2)+1}\n')

The mesh.elements file contains element data arranged as
e1 body type n1 ... nn
e2 body type n1 ... nn
...
en body type n1 ... nn
Each line starts with an integer which is used for identifying the element. The integer body defines the
material body which this element is part of. Then the element type code and element nodes are listed. For
example, the element file might start with the following lines:
1 1 404 1 2 32 31
2 1 404 2 3 33 32
3 1 404 3 4 34 33
4 1 404 4 5 35 34

In [21]:
import numpy as np
tri = np.zeros((mesh.GetNumberOfCells(),3))

for i in range(mesh.GetNumberOfCells()):
    for j in range(3):
        tri[i,j] = mesh.GetCell(i).GetPointIds().GetId(j)

In [22]:
def find_triangles(p1_id, p2_id, tri):
    tt = (tri-p1_id) * (tri-p2_id)
    
    return np.where((tt==0).sum(axis=1)==2)[0]

In [23]:
loc = vtk.vtkPointLocator()
loc.SetDataSet(mesh)
loc.BuildLocator()


with open('mesh.boundary','w') as f:
        #PVs
        f.write(f'1 1 {5892+1} {5506+1} 202 {3743+1} {4890+1}\n')
        f.write(f'2 1 {5892+1} {5249+1} 202 {4890+1} {3938+1}\n')        
        f.write(f'3 1 {5892+1} {4613+1} 202 {3938+1} {3743+1}\n')        
        
        f.write(f'1 2 {2921+1} {4299+1} 202 {3126+1} {2841+1}\n')
        f.write(f'2 2 {2921+1} {3222+1} 202 {2841+1} {3314+1}\n')
        f.write(f'3 2 {2921+1} {4636+1} 202 {3314+1} {3126+1}\n')

        f.write(f'1 3 {15798+1} {16976+1} 202 {8671+1} {8682+1}\n')
        f.write(f'2 3 {15798+1} {16258+1} 202 {8682+1} {8668+1}\n')
        f.write(f'3 3 {15798+1} {16260+1} 202 {8668+1} {8671+1}\n')
        
        f.write(f'1 4 {17762+1} {17761+1} 202 {9125+1} {8928+1}\n')
        f.write(f'2 4 {17762+1} {18030+1} 202 {8928+1} {8855+1}\n')
        f.write(f'3 4 {17762+1} {18054+1} 202 {8855+1} {9125+1}\n')
        
        #top point
        f.write(f'1 5 {14435+1} {13498+1} 202 {7499+1} {5675+1}\n')
        f.write(f'2 5 {14435+1} {12061+1} 202 {5675+1} {7558+1}\n')
        f.write(f'3 5 {14435+1} {14611+1} 202 {7558+1} {7499+1}\n')

        #MV
        for i in range(lines.GetNumberOfCells()):
            ids = lines.GetCell(i).GetPointIds() 
            id1 = loc.FindClosestPoint(lines.GetPoint(ids.GetId(0)))
            id2 = loc.FindClosestPoint(lines.GetPoint(ids.GetId(1)))
            cellids = find_triangles(id1, id2, tri)
            f.write(f'{i+1} 6 {cellids[0]+1} 0 202 {id1+1} {id2+1}\n')
            

The elements that form the boundary are listed in the file mesh.boundary. This file is similar to the
mesh.elements file and is organized as
e1 bndry p1 p2 type n1 ... nn
e2 bndry p1 p2 type n1 ... nn
...
en bndry p1 p2 type n1 ... nn
The first integer is again the identification number of the boundary element. Next the identification number
of the part of the boundary where this element is located is given. Whether the boundary element can be
represented as the side of a parent element defined in the file mesh.elements is indicated using the two
parent element numbers p1 and p2. If the boundary element is located on an outer boundary of the body,
it has only one parent element and either of these two integers may be set to be zero. It is also possible that
both parent element numbers are zeros. Finally the element type code and element nodes are listed.